In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
from sklearn.preprocessing import Imputer
from sklearn.neighbors import NearestNeighbors
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns
import glob
from natsort import natsorted, ns
import impyute as impy
import math
from statistics import mean 
sns.set_style('whitegrid')

In [2]:
#Reading groundtruth file
file_name = r"C:\Users\kshah3\Documents\Kartik IMP\CSC591 ML\Ap1\AP1 Materials and Files-20190829\train_data\train_data\train_groundtruth\{}.csv"
df_list = []
for i in range(1, 6001):
    pt=pd.read_csv(file_name.format(i))
    pt['pt_no'] = i
    pt['index'] = range(1,len(pt)+1)
    df_list.append(pt)
df_truth = pd.concat(df_list)

In [14]:
#Mean imputation
def impute_simple_mean(df_train_column):
    df_train_column = df_train_column.fillna(df_train_column.mean())
    return df_train_column

In [72]:
#Bfill imputation
def impute_bfill(df_train_column):
    if(math.isnan(df_train_column.loc[len(df_train_column)-1]) == True ):
        if(math.isnan(df_train_column.loc[len(df_train_column)-2]) == True ):
            if(math.isnan(df_train_column.loc[len(df_train_column)-3]) == True ):
                if(math.isnan(df_train_column.loc[len(df_train_column)-4]) == True ):
                    if(math.isnan(df_train_column.loc[len(df_train_column)-5]) == True ):
                        if(math.isnan(df_train_column.loc[len(df_train_column)-6]) == True ):
                            df_train_column.loc[len(df_train_column)-6] = df_train_column.loc[len(df_train_column)-7]
                        df_train_column.loc[len(df_train_column)-5] = df_train_column.loc[len(df_train_column)-6]    
                    df_train_column.loc[len(df_train_column)-4] = df_train_column.loc[len(df_train_column)-5]
                df_train_column.loc[len(df_train_column)-3] = df_train_column.loc[len(df_train_column)-4]     
            df_train_column.loc[len(df_train_column)-2] = df_train_column.loc[len(df_train_column)-3]
        df_train_column.loc[len(df_train_column)-1] = df_train_column.loc[len(df_train_column)-2]
    df_train_column = df_train_column.fillna(df_train_column.bfill())
    if(df_train_column.isnull().sum()>0):
        print(df_train_column)
    return df_train_column

In [87]:
#Kartik mean 4 imputation
def impute_fourmean(df_train_column):
    column_datas = []
    column_datas = list(df_train_column)
    empty = []
    elements = []
    final_elements = []
    idx = 0
    count = 0
    for column_data in column_datas:
        elements.append(column_data)
        if  math.isnan(column_data):
            empty.append(idx)
        if(count==3 and (len(empty)-1)<3):
            if(empty!=[]):
                for empty_index in empty:
                    #print("K")
                    #print(empty_index)
                    column_datas[empty_index] = np.nanmean(elements)
            count = -1
            elements = []
            empty = []
        if(count==3 and (len(empty))==4):
            if(empty!=[]):
                for empty_index in empty:
                    #print("element")
                    #print(empty_index)
                    column_datas[empty_index] = np.nanmean(column_datas)
            count = -1
            elements = []
            empty = []       
        if(count<3 and idx == len(column_datas)-1):
            if(empty!=[]):
                for empty_index in empty:
                        last_elements = []
                        last_elements.append(column_datas[idx])
                        last_elements.append(column_datas[idx-1])
                        last_elements.append(column_datas[idx-2])
                        last_elements.append(column_datas[idx-3])
                        #print(last_elements)
                        column_datas[empty_index] = np.nanmean(last_elements)
        final_elements.append(column_data)
        idx+=1
        count+=1
#    cld = pd.DataFrame(column_datas)
#     cld = cld.iloc[0:, 0]
#     columns = ['Potassium']
#     cld.columns=columns
    return pd.Series(column_datas)

In [79]:
dir_name_fourmean = r"C:\Users\kshah3\Documents\Kartik IMP\CSC591 ML\Ap1\AP1 Materials and Files-20190829\train_data\train_data\train_with_fourmean"

In [6]:
dir_name_bfill = r"C:\Users\kshah3\Documents\Kartik IMP\CSC591 ML\Ap1\AP1 Materials and Files-20190829\train_data\train_data\train_with_bfill"

In [88]:
#Reading missing file and imputing it
columns = ['time', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13']
file_name = r"C:\Users\kshah3\Documents\Kartik IMP\CSC591 ML\Ap1\AP1 Materials and Files-20190829\train_data\train_data\train_with_missing\{}.csv" # use your path
df_list = []
for i in range(1, 6001):
    pt=pd.read_csv(file_name.format(i))
    for index in range(pt.shape[1]):
        # Select column by index position using iloc[]
        columnSeriesObj = pt.iloc[: , index]
        re = impute_fourmean(columnSeriesObj)
        df_list.append(re.tolist())
    df_train = pd.DataFrame(df_list).T
    df_train.columns= columns
    df_csv = df_train.to_csv("{}\{}.csv".format(dir_name_fourmean,i),mode = 'w', index=False)
    df_list = []

In [8]:
df_csv = df_train.to_csv('test.csv',mode = 'w', index=False)

In [9]:
#Reading naidx file
file_name =  r"C:\Users\kshah3\Documents\Kartik IMP\CSC591 ML\Ap1\AP1 Materials and Files-20190829\train_data\train_data\naidx.csv"
df_naidx=pd.read_csv(file_name)
len_naidx=len(df_naidx)

In [89]:
fourmean = r"C:\Users\kshah3\Documents\Kartik IMP\CSC591 ML\Ap1\AP1 Materials and Files-20190829\train_data\train_data\train_with_fourmean\{}.csv"
df_list_mice = []
for i in range(1, 6001):
    pt=pd.read_csv(fourmean.format(i))
    pt['pt_no'] = i
    pt['index'] = range(1,len(pt)+1)
    df_list_mice.append(pt)
imputed_val = pd.concat(df_list_mice)

In [90]:
#RMSD per column
d=[-1]*13
diff=0
for i in range(1,14):
    df_naidx_sub=df_naidx.loc[df_naidx['test'] == ('X'+str(i))]
    len_naidx_per_col=len(df_naidx_sub)
    diff=0
    for index, row in df_naidx_sub.iterrows():
        x= imputed_val.loc[(imputed_val['index'] == row['i']) & (imputed_val['pt_no'] == row['pt.num']) ,row['test']].iloc[0]
        y= df_truth.loc[(df_truth['index'] == row['i']) & (df_truth['pt_no'] == row['pt.num']),row['test']].iloc[0]
        df_truth_sub=df_truth.loc[(df_truth['pt_no'] == row['pt.num'])]
        rnge=df_truth_sub[row['test']].max()-df_truth_sub[row['test']].min()
        diff+=((x-y)/rnge)**2
    d[i-1]=math.sqrt(diff/len_naidx_per_col)
    
print (d)

[0.23773964545755322, 0.2617864932491645, 0.24291340172067272, 0.2477396712018247, 0.2520984569446383, 0.25578316905419274, 0.27357313687687596, 0.21840169199273932, 0.2573485792251647, 0.23859827411428927, 0.21833243825388332, 0.24406369773142914, 0.2795095996498526]
